In [1]:
from grading_tools import check, auto_marking_message

# Exercises

## Exercise

You should use the ${\mathbb Q}$-measure for only one purpose. What is that purpose?

 1. Computing expected utilities
 2. Calculating prices
 3. Computing Value at Risk
 4.  Computing Expected Shortfall
 5.  Testing whether delta-hedging works in practice.
    

In [2]:
### BEGIN SOLUTION
2
### END SOLUTION

2

In [3]:
check('53788e',2)
auto_marking_message()

Auto marking message: ✨ Correct


## Exercise

In the lecture notes it is claimed that the solution of the SDE
$$
d \tilde{S}_t = \tilde{S}_t(r \, dt + \sigma \, dW^{\mathbb Q}_t)
$$
is
$$
\tilde{S}_t = S_0 \exp((r - \frac{1}{2}\sigma^2) t + \sigma W^{\mathbb Q}_t ).
$$
How do we know this?

## Exercise

In the lecture notes it is claimed that if
$$
d \tilde{S}_t = \tilde{S}_t(r \, dt + \sigma \, dW^{\mathbb Q}_t)
$$
then 
$\tilde{Z}_T:=\log(\tilde{S}_t)$ is normally distributed with mean $A:=\log(S_0) + (r-\frac{1}{2}\sigma^2)T$ and standard deviation
$B:=\sigma \sqrt{T}$. How do we know this?

## Exercise

A digital call option with strike $K$ and maturity $T$ pays out $1$ if $S_T>K$ and $0$ otherwise.
Compute an analytic formula for the price of a digital call option in the Black-Scholes model
by following the same procedure we used to compute the Black-Scholes formula.

Use this to write a function `black_scholes_digital_call_price`.



In [4]:
### SOLUTION

import scipy.stats

from sympy import *
import math

r, z = symbols('r z', real=True)
K, T, S, sigma = symbols('K T S sigma', real=True, positive=True)
B = sigma * sqrt(T)
A = log(S) + (r-sigma**2/2)*T
# The change in the calculation is that we change the function $f$ in the integrand. 
integrand = exp(-r*T)*(1/(B*sqrt(2*pi)))*exp(-(z-A)**2/(2*B**2))
V = integrate( integrand, (z,log(K),oo) ).simplify()
N = Function('N')
w = Wild('w')
V = V.replace(erf(w),2*N(sqrt(2)*w)-1)
V = V.simplify()
V

N((2*T*r - T*sigma**2 + log(S**2/K**2))/(2*sqrt(T)*sigma))*exp(-T*r)

In [5]:
def black_scholes_digital_call_price(S,r,sigma,K,T):
    # BEGIN SOLUTION
    d2 = 1/(sigma*math.sqrt(T))*(math.log(S/K)+(r-0.5*sigma**2)*T)
    return scipy.stats.norm.cdf(d2)*exp(-T*r)
    # END SOLUTION

In [6]:
check('07bd76', black_scholes_digital_call_price(100,0.05,0.2,105,2))
auto_marking_message()

Auto marking message: 🌟 Correct


## Exercise

Last week we made a dubious argument to justify an estimate of the price of a call option on the top boundary.

We noted that if we wish to price a call option in the Black-Scholes model and $S_t$ is much greater than $K$, then
the probability that $S_T\leq K$ will be negligible. This was true, but it didn't really tell us anything about the price of the option. This is because we were talking about the ${\mathbb P}$-measure probability. What we really needed to know was that the ${\mathbb Q}$-measure probability that $\tilde{S}_T\leq K$ is negligible, but this will also be true.

We can then argue that
$$
\begin{split}
V &= E^{\mathbb Q}_t(e^{-r(T-t)} \max\{\tilde{S}_T-K,0\}) \\
&\approx E_{\mathbb Q}(e^{-r(T-t)} (\tilde{S}_T-K)) \\
&=\tilde{S}_t - e^{-r(T-t)} K.
\end{split}
$$

Justify the final line of this computation.


### Solution

One approach is to note that we can compute this expectation by brute-force integration. Indeed, we did this earlier in the course
when we computed the mean of a process following discrete-time geometric Brownian motion, so we've already computed that
$$
E^{\mathbb Q}(\tilde{S}_T)=e^{rT}(\tilde{S}_0)
$$
and hence
$$
E^{\mathbb Q}_t(\tilde{S}_T)=e^{r(T-t)}(\tilde{S}_t).
$$

Another approach is to note that the stock is itself a derviative with payoff function $f(S_T)=S_T$. We know the price of the stock at time $t$ is $S_t$, but the theory of risk-neutral pricing tells us that the price is also given by
$$
E^{\mathbb Q}_t(e^{-r(T-t)} f(\tilde{S}_T))=E^{\mathbb Q}_t(e^{-r(T-t)} \tilde{S}_T ).
$$
So we can write down that
$$
S_t=E^{\mathbb Q}_t(e^{-r(T-t)} \tilde{S}_T).
$$